## Создание дашборда Яндекс.Дзен

Вы работаете аналитиком в Яндекс.Дзене. Почти всё ваше время занимает анализ пользовательского взаимодействия с карточками статей.
Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».
Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».
Есть три способа взаимодействия пользователей с системой:
Карточка отображена для пользователя (show);
Пользователь кликнул на карточку (click);
Пользователь просмотрел статью карточки (view).
Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы: 
Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
Как много карточек генерируют источники с разными темами?
Как соотносятся темы карточек и темы источников?
На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.
Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан для вас дата-инженерами.

Источники данных для дашборда: дата-инженеры обещали подготовить для вас агрегирующую таблицу dash_visits. 

Вот её структура:
* record_id — первичный ключ,
* item_topic — тема карточки,
* source_topic — тема источника,
* age_segment — возрастной сегмент,
* dt — дата и время,
* visits — количество событий.

Частота обновления данных: один раз в сутки, в полночь по UTC;
Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):


In [1]:
from IPython.display import Image #макет дашборда
Image(url='https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-06T161108.832_1594041107.png')

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

In [3]:
query = '''
           SELECT *
           FROM dash_visits
        '''

dash_visits = pd.io.sql.read_sql(query, con=engine)

In [4]:
dash_visits.head() # смотрим файл

record_id item_topic source_topic age_segment                  dt  visits
0    1040597     Деньги         Авто       18-25 2019-09-24 18:32:00       3
1    1040598     Деньги         Авто       18-25 2019-09-24 18:35:00       1
2    1040599     Деньги         Авто       18-25 2019-09-24 18:54:00       4
3    1040600     Деньги         Авто       18-25 2019-09-24 18:55:00      17
4    1040601     Деньги         Авто       18-25 2019-09-24 18:56:00      27

In [5]:
dash_visits.info() # пропусков не видим

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [8]:
print('Количество дубликатов: ',dash_visits.duplicated().sum()) # проверим дубликаты

Количество дубликатов:  0


In [9]:
dash_visits['item_topic'].value_counts() # посмотрим названия топиков

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

In [10]:
dash_visits['source_topic'].value_counts() # посмотрим темы источника

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

In [11]:
dash_visits['age_segment'].value_counts() # посмотрим на возрастные группы

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

In [12]:
dash_visits.to_csv('dash_visits.csv', index = False) # переводим в формат csv

## Дашборд
https://public.tableau.com/views/Shilyaev_Tablo/sheet4?:language=en-US&:display_count=n&:origin=viz_share_link

## Презентация
https://disk.yandex.ru/i/L38ej_9Vc5VuQ


## Выводы
Наиболее популярные темы источников:
Семейные отношения (10-11% посещений)
Россия (9-10% посещений)
Полезные советы (8-9% посещений)
Путешествия (7-8% посещений)
Знаменитости (7-8% посещений)

Наименее популярные темы источников:
Финансы (0.8-1% посещений)
Музыка (0.8-1% посещений)
Строительство (0.8-1% посещений)
Технологии (1% посещений)
Интерьеры (1.24% посещений)

Наиболее популярные темы карточек:
Наука (4.372 посещения)
Отношения (4.145 посещений)
Интересные факты (3.910 посещений)
Общество (3.723 посещений)
Подборки (3.520 посещений)

Наименее популярных темы карточек:
Знаменитости (1.463 посещения)
Шоу (1.485 посещений)
Женская психология (1.518 посещений)
Психология (1.538 посещений)
Искусство (1.707 посещений)

Наиболее популярные отношения источника к карточке:
Путешествия - Рассказы (4.587 посещений)
Россия - Общество (3.471 посещение)
Кино - Наука (3.279 посещений)
Россия - Россия (2.847 посещений)
Полезные советы - Подборки (2.795 посещений)
